In [495]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p

In [496]:
min_max_scaler = p.MinMaxScaler()

INPUT_DATA_PATH = 'test_input.npy'
OUTPUT_DATA_PATH = 'test_output.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

# print(x_input)

In [497]:
# x_input[x_input < 1] = 0


# print(x_input[0][0])

In [498]:
def threshold(tensor):
    for img in range(len(tensor)):
            for channel in range(1):
                    for h in range(100):
                            for w in range(100):
                                    if (tensor[img][channel][h][w] < 1):
                                            tensor[img][channel][h][w] = 0

In [499]:
SPLIT_IDX = 15 #750
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

threshold(x_train)                       
threshold(x_test)
threshold(y_train)                       
threshold(y_test)

class LinesDataset(TensorDataset):
        def __init__(self, lines, labels):
                self.lines = lines
                self.labels = labels
        def __len__(self):
                return len(self.lines)
        def __getitem__(self, idx):
                line = torch.tensor(self.lines[idx])
                label = torch.tensor(self.labels[idx])
                sample = {"Line": line, "Label": label}
                return sample


train_dataset = TensorDataset(x_train,y_train) # create your datset
# train_dataloader = DataLoader(train_dataset, shuffle=True) # create your dataloader

train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)

#/home/bwilab/asha_ritu/line_model/conv_network.ipynb


3


In [500]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            # first layer: learning one person. 
            # kernel size = diameter of a dot (5 pixels) + padding (2 pixels) = (7,7)
            # out_channels: 1 for now
            # try changing the out_channels
            # larger dataset and ReLU between each layer
            # larger dataset, more layers, larger kernels

            # find center pixel of each circle, count number of dots

            # blob detect: looking for continuous regions of non-white pixels
            # confidence regions. 
            # masking the image like in hw3
            # 
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(6,6), stride=1, bias=True, padding=(2,2)), # input layer
            # nn.ReLU(),

            # layer 2: learning two dots.
            # kernel_size = diameter of two dots (10 pixels) + distance between them (2 pixels) + padding (2 pixels)
            # there are 8 different orientations two people can be in, resulting in 8 output channels.
            # two options: either increase kernel size and number of channels or
            # pool -> lower image resolution and learn a smaller pattern

            # nn.MaxPool2d()

            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(14,14), stride=1, bias=True, padding=(7,7)),
            # nn.ReLU(),
            # final layer: out_channels=1, 
            # kernel size = ? ask Dr. Hart what a reasonable kernel size for the output layer could be.
            nn.Conv2d(in_channels=8, out_channels=1, kernel_size=(13,13), padding=(6,6)),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [501]:
model = NeuralNetwork()
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size = 4
epochs = 5
# Initialize the loss function
loss_fn = nn.MSELoss()
# loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
test_losses = []

In [502]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        # print(X)
        pred = model(X)
        # print(f"pred:{pred}")
        # print(f"target:{y}")
    
        # print(f"pred: {pred}")
        # print(f"y shape:{y.shape}")
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        
        print(f"loss: {loss:>7f}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# def test_loop(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss, correct = 0, 0

#     with torch.no_grad():
#         for X, y in dataloader:
#             # print(X.shape)
#             pred = model(X)
#             # print(pred.argmax(1).shape)
#             # print(y.shape)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred == y).type(torch.float).sum().item()
#             # print(correct)

#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [503]:
# def test():
#   model.eval()
#   test_loss = 0
#   correct = 0
#   avg_accuracy = 0
#   with torch.no_grad():
#     for data, target in test_dataloader:
#       output = model(data)
#       test_loss += loss_fn(output, target).item()

#       img = 0
#       h = 0
#       w = 0        

#       # compare the number of dots, not the pixel by pixel comparison

#       for img in range (5):
#         correct = 0
#         # print (target[img][0].shape)
#         for h in range (100):
#           for w in range (100):
#             # print(f"output {output[img][0]}")
#             # print(f"target {target[img][0]}")
#             if (target[img][0][h][w] == 0 and output[img][0][h][w] == 0):
#               correct += 1

        # print(f"percentage of pixels correct for image {img}: {correct/10000}")

        # print (f"num correct: {correct}")

      # num_correct = output.eq(target.data.view_as(output)).sum()
      
  # test_loss /= len(test_dataloader.dataset)
  # test_losses.append(test_loss)
  # print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  #   test_loss, correct, len(test_dataloader.dataset),
  #   100. * correct / len(test_dataloader.dataset)))

  # avg_accuracy /= 1000
  # print(f"average accuracy: {avg_accuracy}")



In [504]:
# test()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test()
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")


Epoch 1
-------------------------------
loss: 0.989012
loss: 0.978897
loss: 0.820703
Epoch 2
-------------------------------
loss: 0.180169
loss: 2.571131
loss: 0.992520
Epoch 3
-------------------------------
loss: 0.993140
loss: 0.992720
loss: 0.993680
Epoch 4
-------------------------------
loss: 0.992680
loss: 0.993600
loss: 0.993260
Epoch 5
-------------------------------
loss: 0.993640
loss: 0.992760
loss: 0.993140
Done!


In [507]:
model = NeuralNetwork()
with torch.no_grad():
    for data, target in test_dataloader:

        i = 2
        # print (target[0][0])
        output = model(data)
        # print(output)
        # print(f"{output[4][0]}")
        # with open('model_output.npy', 'wb') as f:
        #     np.save(f, output)

        # INPUT
        input_arr = np.array(data[i][0])*255
        input = Image.fromarray(input_arr)
        input.show()
        

        # ACTUAL OUTPUT
        output = [(1-x) for x in output]
        arr = np.array(output[i][0])
        arr[arr < 1] = 0
        arr = arr * 255
        im = Image.fromarray(arr)
        im.show()

        # EXPECTED OUTPUT
        expected_arr = np.array(target[i][0]) * 255
        # expected_arr[expected_arr < 1] = 0
        # expected_arr = expected_arr * 255
        # im_expected = Image.fromarray(expected_arr)
        # im_expected.show()
        

       
        break